# Registro de modelos y versionado con MLFlow

In [12]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging
mlflow.set_tracking_uri("sqlite:///mlruns.db")
#mlflow.set_tracking_uri("http://127.0.0.1:5000/")
#mlflow.set_tracking_uri("sqlite:///mydb.sqlite")

mlflow.set_experiment("Wine_prediction")

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2023/05/09 15:30:26 INFO mlflow.tracking.fluent: Experiment with name 'Wine_prediction' does not exist. Creating a new experiment.


In [8]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


In [16]:
if __name__ == "__main__":
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )
    try:
        data = pd.read_csv(csv_url, sep=";")
        
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    alpha = 0.5
    l1_ratio = 0.5

    with mlflow.start_run():
        
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        #mlflow.sklearn.log_model(lr, "model")

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
        else:
            mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.793164022927685
  MAE: 0.6271946374319586
  R2: 0.10862644997792636


Registered model 'ElasticnetWineModel' already exists. Creating a new version of this model...
2023/05/09 15:34:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetWineModel, version 4
Created version '4' of model 'ElasticnetWineModel'.


In [4]:
!mlflow server --registry-store-uri 'sqlite:///mlruns.db'

2023/05/09 15:16:22 ERROR mlflow.cli: Error initializing backend store
2023/05/09 15:16:22 ERROR mlflow.cli: [WinError 3] El sistema no puede encontrar la ruta especificada: 'E:\\'
Traceback (most recent call last):
  File "C:\Users\alexb\anaconda3\lib\site-packages\mlflow\cli.py", line 404, in server
    initialize_backend_stores(backend_store_uri, registry_store_uri, default_artifact_root)
  File "C:\Users\alexb\anaconda3\lib\site-packages\mlflow\server\handlers.py", line 280, in initialize_backend_stores
    _get_model_registry_store(registry_store_uri)
  File "C:\Users\alexb\anaconda3\lib\site-packages\mlflow\server\handlers.py", line 271, in _get_model_registry_store
    _model_registry_store = _model_registry_store_registry.get_store(store_uri)
  File "C:\Users\alexb\anaconda3\lib\site-packages\mlflow\tracking\_model_registry\registry.py", line 43, in get_store
    return self._get_store_with_resolved_uri(resolved_store_uri, resolved_tracking_uri)
  File "C:\Users\alexb\anaconda3